In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import warnings
import datetime
import argparse
import threading
import multiprocessing
import os

from tqdm import tqdm

from GBP_Simulations.GBP.data import DataGenerator
from GBP_Simulations.GBP.gbp import run_GaBP_SYNC_ACCELERATED, run_GaBP_SYNC_ACCELERATED_UPDATED, run_GaBP_HARDWARE_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL
from GBP_Simulations.GBP.utilities import HiddenPrints
from GBP_Simulations.GBP.visulisation import set_plot_options, get_plot_colors, NetworkxGraph, AnalyzeResult

# Option 1: Suppress all warnings
warnings.filterwarnings("ignore")

set_plot_options()
colors = get_plot_colors()

data_gen = DataGenerator()
result_analyzer = AnalyzeResult()

### 2D Lattice

In [2]:
x = 100
y = 100

A,b = data_gen.get_2D_lattice_matrix_PSD(x,y,scaling=False)

In [3]:
convergence_type = 'all' #all
simulation_convergence_threshold = 1*10**-10

P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, 
                                                                                                      b, 
                                                                                                      max_iter=1_000, 
                                                                                                      mae=True if convergence_type == 'mae' else False, 
                                                                                                      convergence_threshold=simulation_convergence_threshold,
                                                                                                      show=True)
final_mean_sync = list(mu_i)

iteration: 1
0.027223216658180227
-----
iteration: 2
0.0008151821819333547
-----
iteration: 3
0.0001282128898667428
-----
iteration: 4
2.0434694917617863e-05
-----
iteration: 5
3.15076588650757e-06
-----
iteration: 6
4.547541895367085e-07
-----
iteration: 7
6.384656081662504e-08
-----
iteration: 8
8.981349655420477e-09
-----
iteration: 9
1.3170890159654386e-09
-----
iteration: 10
3.2927225399135965e-10
-----
iteration: 11
0.0
-----


In [4]:
print(A.shape)

(1024, 1024)


### Hardware

In [5]:
ASYNC_ITER = 10
NODE_UPDT_PE = 100
PEs = 1

async_convergence_threshold = 1*10**-5

In [6]:
sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean_sync, max_iter=1000, mae=False, convergence_threshold=async_convergence_threshold, show=True)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 1
0.01691336975311108
-----
iteration: 2
0.01691336975311108
-----
iteration: 3
0.01691336975311108
-----
iteration: 4
0.01691336975311108
-----
iteration: 5
0.014269161032058168
-----
iteration: 6
0.014269161032058168
-----
iteration: 7
0.014247735824189402
-----
iteration: 8
0.014247735824189402
-----
iteration: 9
0.014247735824189402
-----
iteration: 10
0.014247735824189402
-----
iteration: 11
0.014247735824189402
-----
iteration: 12
0.014247735824189402
-----
iteration: 13
0.014247735824189402
-----
iteration: 14
0.014247735824189402
-----
iteration: 15
0.014247735824189402
-----
iteration: 16
0.014247735824189402
-----
iteration: 17
0.014247735824189402
-----
iteration: 18
0.014247735824189402
-----
iteration: 19
0.014247735824189402
-----
iteration: 20
0.014247735824189402
-----
iteration: 21
0.014247735824189402
-----
iteration: 22
0.014247735824189402
-----
iteration: 23
0.014247735824189402
-----
iteration: 24
0.014247735824189402
-----
iteration: 25
0.0142477358241